In [1]:
!pip install langchain_huggingface
!pip install langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is

In [3]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
llm = HuggingFacePipeline.from_model_id(
        model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        task = "text-generation",
        pipeline_kwargs = dict(
            temperature = 0.1
        )
    )
model = ChatHuggingFace(llm=llm)

2025-07-08 18:59:54.351942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752001194.704436      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752001194.807117      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00:00:0100:01


In [6]:
!pip install langchain

In [11]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_GEMMA_READ_KEY")

In [12]:
import os
from huggingface_hub import login

login(token=secret_value_0)

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Load tokenizer and model
model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
# Create HF pipeline
hf_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.1,
    do_sample=True,
    top_k=50,
    top_p=0.95
)

# Wrap with LangChain HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipe)
chat_model = ChatHuggingFace(llm=llm)

Device set to use cuda:0


In [25]:
from pprint import pprint

In [26]:
pprint(chat_model.invoke("Write a report on black hole"))

AIMessage(content="<bos><start_of_turn>user\nWrite a report on black hole<end_of_turn>\n<start_of_turn>model\n**Black Hole**\n\n**Definition:**\n\nA black hole is a region of spacetime with such strong gravitational pull that nothing, not even light, can escape its gravitational pull. It is a region of spacetime with a gravitational field so strong that it causes spacetime itself to curve around it.\n\n**Formation:**\n\nBlack holes are formed when a massive star exhausts its nuclear fuel and collapses under its own gravity. This can happen when a star with a mass several times that of the Sun collapses to a radius a few times that of the Sun, or when a galaxy's central black hole grows by absorbing matter from its surroundings.\n\n**Properties:**\n\n* **Event horizon:** The event horizon is the boundary around a black hole beyond which nothing can escape. It is the point of no return.\n* **Singularity:** The singularity is the point at the center of a black hole at which the gravitatio

## String Output Parser - StrOutputParser

topic --> LLM --> report --> LLM --> 5line summarize

In [27]:
from langchain_core.prompts import PromptTemplate

In [28]:
#Detailed prompt
template1 = PromptTemplate(
    template = "Write a detailed report on {topic}",
    input_variables = ["topic"]
)

#Summarize prompt
template2 = PromptTemplate(
    template = "Write 3 lines summary on the following text. /n {text}",
    input_variables = ["text"]
)


In [30]:
prompt1 = template1.invoke({"topic": "black hole"})
result = chat_model.invoke(prompt1)
print(result.content)

<bos><start_of_turn>user
Write a detailed report on black hole<end_of_turn>
<start_of_turn>model
**Black Hole**

A black hole is a region of spacetime with such strong gravitational pull that nothing, not even light, can escape its gravitational pull. It is a region of spacetime with a region of infinite density and zero volume.

**Definition**

A black hole is a region of spacetime in which gravity is so strong that nothing, not even light, can escape. This is because the gravitational force of a black hole is so strong that it creates a region of spacetime in which nothing can escape, not even light.

**Formation**

Black holes are formed when a massive star exhausts its nuclear fuel and collapses under its own gravity. When a star exhausts its nuclear fuel, it can no longer support its own weight and begins to collapse. If the star is massive enough, it will reach a point where its gravitational pull is strong enough to create a black hole.

**Event Horizon**

The event horizon is t

In [31]:
prompt2 = template2.invoke({"text": result.content})
result_summary = chat_model.invoke(prompt2)
print(result_summary.content)

<bos><start_of_turn>user
Write 3 lines summary on the following text. /n <bos><start_of_turn>user
Write a detailed report on black hole<end_of_turn>
<start_of_turn>model
**Black Hole**

A black hole is a region of spacetime with such strong gravitational pull that nothing, not even light, can escape its gravitational pull. It is a region of spacetime with a region of infinite density and zero volume.

**Definition**

A black hole is a region of spacetime in which gravity is so strong that nothing, not even light, can escape. This is because the gravitational force of a black hole is so strong that it creates a region of spacetime in which nothing can escape, not even light.

**Formation**

Black holes are formed when a massive star exhausts its nuclear fuel and collapses under its own gravity. When a star exhausts its nuclear fuel, it can no longer support its own weight and begins to collapse. If the star is massive enough, it will reach a point where its gravitational pull is strong 

## Using StrOutputParser

In [32]:
!pip install langchain_core

In [33]:
from langchain_core.output_parsers import StrOutputParser

In [34]:
parser = StrOutputParser()

### Above can be shorted using chain

In [36]:
chain = template1 | chat_model | parser | template2 | chat_model | parser
result = chain.invoke({"topic": "black hole"})
print(result)

<bos><start_of_turn>user
Write 3 lines summary on the following text. /n <bos><start_of_turn>user
Write a detailed report on black hole<end_of_turn>
<start_of_turn>model
**Black Hole**

**Definition:**

A black hole is a region of spacetime with such strong gravitational pull that nothing, not even light, can escape its gravitational pull. It is a region of spacetime with a gravitational field so strong that it causes spacetime itself to curve around it.

**Formation:**

Black holes are formed when a massive star exhausts its nuclear fuel and collapses under its own gravity. This process is known as gravitational collapse. When a star exhausts its nuclear fuel, it can no longer support its own weight and gravitational forces become so strong that it collapses into a black hole.

**Characteristics:**

* **Event horizon:** The event horizon is the boundary around a black hole beyond which nothing can escape. It is the point of no return.
* **Singularity:** The singularity is the point at